In [1]:
import pickle

import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict

from pymatgen.ext.matproj import MPRester
from pymatgen.core.composition import Composition
from pymatgen.core.periodic_table import Element
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter

from matminer.featurizers.site import CrystalNNFingerprint
from matminer.featurizers.structure import SiteStatsFingerprint

# Initialize the MP Rester
mpr = MPRester('pn8XdbGhMrv90STu')


In [2]:
def pourbaix_data_multielement(pbx, element1, element2, ph, voltage, entries_of_interest):
    """
    Arguments:
    pbx:           the PourbaixDiagram inputs, passed to the function as 
                   entries = mpr.get_pourbaix_entries([metal])
                   pbx_dat = PourbaixDiagram(entries, filter_solids=True)
                   pbx_dat would then be passed in as the argument.
    entries:       A list of Pourbaix Entries, a pymatgen data type of each materials project datapoint considered in 
                   the Pourbaix diagram
    element1:        One of the metals to consider, passed as a string ie. "Mn"
    element2:        Other metal to consider, passed as a string ie. "Mn". By default, we set metal2=None, so that we can
                   decide when we are using the function whether we want to use one or two metals.

    ph:            A single value or range of values
    voltage:       A single value or range of values (must match ph in shape)
    comp_entries:  Since the PourbaixDiagram object has to be redrawn if there is a change in compostion, 
                   we pre-sort the entries by composition so that we don't have to do them for each entry but for 
                   each given composition.

    
    Returns:
    material_names:     List of names of each material which are considered stable by the function, as a string.
    materials_proj_ids: List of MP ids of each material which are considered stable by the function, as a string.
    energies:           List of ΔG_pbx of each material thats considered stable by the function, as floating point.
    """

    materials_proj_ids = []
    material_names = []
    energies = []
    
    for entry in entries_of_interest:
        energy = pbx.get_decomposition_energy(entry, pH=ph, V=voltage)
        #print(energy)
        materials_proj_ids.append(entry.entry_id)
        material_names.append(entry.name)
        energies.append(energy)
        
    return material_names, materials_proj_ids, energies


In [8]:
binary_oxide_data = pickle.load(open("binary_oxide_data_form_ene.p", "rb"))
binary_oxide_data['Mn_Sb']

{'names': ['Mn(SbO3)2(s)',
  'Mn(SbO2)2(s)',
  'Mn(SbO3)2(s)',
  'Mn(SbO3)2(s)',
  'Mn(SbO3)2(s)',
  'Mn3SbO8(s)',
  'Mn3SbO8(s)',
  'Mn3SbO8(s)',
  'MnSbO4(s)',
  'MnSbO4(s)',
  'MnSbO4(s)',
  'MnSbO4(s)',
  'Mn2Sb2O7(s)',
  'MnSbO4(s)',
  'Mn2Sb2O7(s)',
  'Mn5Sb3O16(s)',
  'Mn(SbO3)4(s)',
  'Mn5SbO12(s)'],
 'mp_ids': ['mp-763546',
  'mp-19231',
  'mp-22576',
  'mp-690561',
  'mp-1078318',
  'mp-774390',
  'mp-774654',
  'mp-754156',
  'mp-755412',
  'mp-755527',
  'mp-1277420',
  'mp-1280914',
  'mp-1190856',
  'mp-1294322',
  'mp-541741',
  'mp-771391',
  'mp-757065',
  'mp-753773'],
 'energies': [array([[0.01461037, 0.01461037, 0.01461037, 0.01461037, 0.01461037,
          0.0220094 , 0.04889227, 0.10444783, 0.16000338, 0.21555894,
          0.27111449]]),
  array([[0.47087166, 0.52801451, 0.58515737, 0.64230023, 0.69944308,
          0.76609898, 0.85780552, 0.98637695, 1.11494838, 1.24351981,
          1.37209124]]),
  array([[0.00000000e+00, 2.96059473e-16, 0.00000000e+00, 2.9605

In [13]:
#binary_oxide_data = defaultdict()
# Want to fill in this with metals of interest
#binary_oxide_data = pickle.load(open("binary_oxide_data.p", "rb"))
binary_oxide_data = pickle.load("binary_oxide_data_form_ene.p", "rb")
actives = [ "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Nb", "Mo"]
pairers = ["Li", "Be", "Na", "Mg", "K", "Ca", "Rb", "Sr", "Cs", "Ba", 
           "Sc",
           "Zn", "Ga", "Ge", "As", "Se", "Br", "Y", "Se", "Tc", "Cd",
           "Ru", "Rh", "Pd", "Ag",
           "In", "Sn", "Sb", "Te", "I", "Hg", "Tl", "Pb", "Bi", 
          "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au" ]

# Could vary this range, ie. perhaps we want to look at pH up to 2/3
min_ph = 0
max_ph = 0
min_v = 1
max_v = 2

ssf = SiteStatsFingerprint(
    CrystalNNFingerprint.from_preset('ops', distance_cutoffs=None, x_diff_weight=0),
    stats=('mean', 'std_dev', 'minimum', 'maximum'))


for idx, ele in enumerate(actives):
    for ele2 in actives[1+idx:]+pairers:
        if ele+"_"+ele2 in binary_oxide_data:
            continue        
        print(ele, ele2)
        # pass the pH and V as an numpy 'meshgrid'.

        pH, V = np.mgrid[
        min_ph : max_ph : 1 * 1j,
        min_v : max_v : 11 * 1j,
        ]
        
        # here, we get all entries in the materials project database with metal/O/H
        entries = mpr.get_pourbaix_entries([ele, ele2])
        # now need to sort entries by composition so that they are read in cleverly
        composition2entries = defaultdict(list)
        for entry in entries:
            if ele in entry.name and ele2 in entry.name and "O" in entry.name and "H" not in entry.name and "ion" not in entry.entry_id:
                composition = entry.composition.as_dict()
                amount_ele1 = composition[ele]
                amount_ele2 = composition[ele2]
                composition1 = amount_ele1/(amount_ele1+amount_ele2)
                composition2entries[str(composition1)].append(entry)
        
        structures = []
        #fingerprints = []
        material_names_ = []
        material_ids_ = []
        energies_ = []
        form_enes_ = []

        for comp in composition2entries:
            pbx_dat = PourbaixDiagram(entries, filter_solids=True, comp_dict={ele: float(comp),
                                                                              ele2: 1-float(comp)})
            
            
            # Then, we use the function in the above cell where we only pass one metal.
            material_names, material_ids, energies = pourbaix_data_multielement(pbx_dat, ele, ele2, 
                                                                                pH, V, composition2entries[comp])
            assert len(material_names)==len(material_ids)==len(energies)
            for idx, mp_id in enumerate(material_ids):
                try:
                    structure = mpr.get_structure_by_material_id(mp_id)
                    # exception can happen below
                    #fingerprint = np.array(ssf.featurize(structure))
                    # exception happens above
                    structures.append(structure)
                    #fingerprints.append(fingerprint)
                    material_names_.append(material_names[idx])
                    material_ids_.append(material_ids[idx])
                    energies_.append(energies[idx])
                    ref_query = mpr.get_entry_by_material_id(material_ids[idx], property_data=['formation_energy_per_atom'])
                    ref_form_ene = ref_query.data['formation_energy_per_atom']
                    form_enes_.append(ref_form_ene)

                except ValueError as e:
                    print(e)
                    

        #assert len(fingerprints)==len(structures)==len(material_ids_)==len(material_names_)==len(energies_)
        assert len(structures)==len(material_ids_)==len(material_names_)==len(energies_)

        binary_oxide_data[ele+"_"+ele2] = {
            "names": material_names_,
            "mp_ids": material_ids_,
            "energies": energies_,
            "structures": structures,
            #"fingerprints": fingerprints,
            "formation_energies": form_enes_
        }
        pickle.dump(binary_oxide_data, open("binary_oxide_data_form_ene.p", "wb"))
    

Ti V


  0%|          | 0/142 [00:00<?, ?it/s]

  0%|          | 0/142 [00:00<?, ?it/s]

  0%|          | 0/142 [00:00<?, ?it/s]

  0%|          | 0/142 [00:00<?, ?it/s]

...

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

Mo Ir


  0%|          | 0/48 [00:00<?, ?it/s]

Mo Pt
Mo Au
